In [ ]:
from langchain_classic.agents import AgentExecutor, create_react_agent
from langchain_classic import hub

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import requests
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("WEATHER_API_KEY")

True

In [7]:
@tool
def search_tool(query: str) -> str:
	"""Search the web."""
	return DuckDuckGoSearchRun().run(query)

In [ ]:
@tool
def get_wheather(city: str) -> str:
	"""Get weather for a given city."""
	url = "http://api.weatherstack.com/current?access_key={API_KEY}&query={city}"
	response = requests.get(url)
	return response.json()

In [2]:
prompt = hub.pull("hwchase17/react")

In [16]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [8]:
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

agent = create_react_agent(
	llm = model,
    tools=[search_tool, get_wheather],
    prompt=prompt
)

In [9]:
agent_executor = AgentExecutor.from_agent_and_tools(
	agent=agent,
	tools=[search_tool, get_wheather],
	verbose=True
)

In [10]:
response = agent_executor.invoke({"input": "what is the capital of india and what is its weather ?"})



> Entering new AgentExecutor chain...
I need to find the capital of India and then find the weather in that capital city.
Action: search_tool
Action Input: "capital of india"Delhi,[b] officially the National Capital Territory (NCT) of Delhi, is a city and a union territory of India containing New Delhi, the capital of India . a b c "The Gazette of India , National Capital Region Planning Board Act of 1985" (PDF) . ... of $370 billion, Delhi-NCR muscles out Mumbai as economic ... The national capital of India , New Delhi is jointly administered by both the Central Government of India and the local Government of Delhi , it is ... In 1858,Prayagraj became the capital of India for a day when it also served as the capital of North-Western Provinces . ... 26 January 1950, a representation of the Lion Capital of Ashoka placed above the motto, Satyameva Jayate , was adopted as the State Emblem of India .The capital of India is New Delhi. Now I need to get the weather in New Delhi.
Action: ge

In [13]:
print(response["output"])

The capital of India is New Delhi and the weather there is Patchy rain nearby with a temperature of 30 degrees Celsius.
